# Number-GAN

* 그냥 noise z 와 input x 에 label y 를 concat 으로 때려박으면 되는 듯함
* G(z) 의 loss 값이 D 에 따라 계산되기 때문에 y 를 따질 필요가 없음. 무슨 말이냐면:
    * y=1 인 G(z) 에서 2를 만들었다고 하자.
    * D(x) 를 계산할 때에도 y 를 때려박기 때문에, G(z) 의 loss 는 D(G(z)+y) 로 계산한다.
    * 이 경우, G(z) 는 2 지만 y 가 1을 나타내기 때문에 D 가 fake 라고 판단할 수 있음 (암만 G(z)가 2를 잘 만들었다고 해도)
    * 따라서 결국 G(z) 가 y 에 해당하는 이미지를 제너레이트하도록 학습이 됨
* 굉장히 소소한 정보로 학습하는 거 같은데, 암튼 잘 되나 확인해보자.
    * 노이즈 d 는 100이고, 라벨 d 는 10 (one-hot)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
D_W1 = tf.Variable(xavier_init([794, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
D_params = [D_W1, D_b1, D_W2, D_b2]

In [4]:
# Z is noised input
# G(z) => generated new x
Z = tf.placeholder(tf.float32, shape=[None, 100])
G_W1 = tf.Variable(xavier_init([110, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))
G_params = [G_W1, G_b1, G_W2, G_b2]

In [5]:
def discriminator(x):
    D_a1 = tf.matmul(x, D_W1) + D_b1
    D_h1 = tf.nn.relu(D_a1)
    # what is logit?
    # logit is inverse function of sigmoid function
    # therefore, D_logit = logit(D_prob).
    # 즉, sigmoid 함수에 들어가는 값이 logit 이라고 할 수 있음.
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    return D_prob, D_logit

In [6]:
def generator(z):
    G_a1 = tf.matmul(z, G_W1) + G_b1
    G_h1 = tf.nn.relu(G_a1)
    # what is differ between logit <> log_prob ?
    # 이것도 걍 로짓이 맞는거 같은데? logit_prob 의 약자일것 같음.
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    
    return G_prob

In [7]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [8]:
def add_label(z, y):
    return tf.concat([z, y], 1)

In [9]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [10]:
G_sample = generator(add_label(Z, Y))
D_real, D_logit_real = discriminator(add_label(X, Y))
D_fake, D_logit_fake = discriminator(add_label(G_sample, Y))

# V1. paper-based learning (다른 코드를 보면 logit 을 이용해서 CE로 계산을 함. 이게 더 성능이 잘 나오나봄)
# http://bamos.github.io/2016/08/09/deep-completion/ 참고

# maximize x => minimize -x
#D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))
#G_loss = -tf.reduce_mean(tf.log(D_fake))

# V2.
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_params)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_params)

In [11]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
batch_size = 100
Z_dim = 100

In [13]:
D_losses = []
G_losses = []

In [15]:
sess = tf.Session()
# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

idx = 0

for it in range(1000000):
    if it % 1000 == 0:
        y = np.zeros([16, 10])
        for i in range(16):
            y[i, i%10] = 1
        samples = sess.run(G_sample, feed_dict={Y: y, Z: sample_Z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('out_num/{:0>4d}.png'.format(idx), bbox_inches='tight')
        idx += 1
        plt.close(fig)
    
    # _ 는 원래 Y 임.
    X_batch, Y_batch = mnist.train.next_batch(batch_size)
    
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_batch, Y: Y_batch, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Y: Y_batch, Z: sample_Z(batch_size, Z_dim)})
    
    if it % 1000 == 0:
        print("[{:7d}] D_loss: {:.4f} / G_loss: {:.4f}".format(it, D_loss_curr, G_loss_curr))
#         D_losses.append(D_loss_curr)
#         G_losses.append(G_loss_curr)

[      0] D_loss: 1.4891 / G_loss: 2.5781
[   1000] D_loss: 0.0116 / G_loss: 9.4070
[   2000] D_loss: 0.0205 / G_loss: 6.4561
[   3000] D_loss: 0.0372 / G_loss: 5.4466
[   4000] D_loss: 0.0468 / G_loss: 6.0719
[   5000] D_loss: 0.1500 / G_loss: 6.3863
[   6000] D_loss: 0.2849 / G_loss: 6.9143
[   7000] D_loss: 0.2389 / G_loss: 4.6980
[   8000] D_loss: 0.3017 / G_loss: 3.8641
[   9000] D_loss: 0.5795 / G_loss: 3.3079
[  10000] D_loss: 0.5581 / G_loss: 3.6674
[  11000] D_loss: 0.5096 / G_loss: 3.0425
[  12000] D_loss: 0.6135 / G_loss: 2.8697
[  13000] D_loss: 0.6491 / G_loss: 2.9981
[  14000] D_loss: 0.8409 / G_loss: 2.6111
[  15000] D_loss: 0.7721 / G_loss: 3.1465
[  16000] D_loss: 0.7877 / G_loss: 2.5332
[  17000] D_loss: 0.6865 / G_loss: 2.0501
[  18000] D_loss: 0.7051 / G_loss: 1.7502
[  19000] D_loss: 0.7695 / G_loss: 2.2789
[  20000] D_loss: 0.8306 / G_loss: 2.3188
[  21000] D_loss: 0.6048 / G_loss: 1.8043
[  22000] D_loss: 0.7720 / G_loss: 2.0681
[  23000] D_loss: 0.7307 / G_loss:

## Discriminator accuracy comparision by number

* 각 숫자에 대해서 discriminator 의 정확도를 비교해보자.
* Vanilla GAN 의 수행 결과가 1로 수렴하는것으로 봐서는 1이 가장 정확해야 함.
* D_fake_sum 은 fake data (generated by G) 에 대한 판단 결과이므로, 높을수록 generator 가 잘 생성한다 (discriminator 를 잘 속인다) 는 의미다.

In [26]:
for num in range(10):
    N = 1000 # 각 숫자당 테스트횟수
    y = np.zeros([N, 10])
    y[:, num] = 1.
    D_fake_res = sess.run(D_fake, feed_dict={Y: y, Z: sample_Z(N, Z_dim)})
    D_fake_sum = np.sum(D_fake_res)
    print("[{}] {}".format(num, D_fake_sum))

[0] 218.727233887
[1] 236.713623047
[2] 127.534362793
[3] 213.932510376
[4] 137.461715698
[5] 173.825912476
[6] 165.678924561
[7] 133.268173218
[8] 187.24105835
[9] 184.819854736
